In [1]:
import zipfile
import os

from keras import utils
import tensorflow as tf

2024-08-20 08:59:21.283405: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-20 08:59:21.283475: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-20 08:59:21.284879: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-20 08:59:21.293119: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-20 08:59:22.350831: W tensorflow/comp

In [2]:
import os
import zipfile

def unzip_to_cwd(zip_filepath, target_folder='train'):
    current_dir = os.getcwd()
    target_folder_path = os.path.join(current_dir, target_folder)

    if not os.path.exists(target_folder_path):
        os.makedirs(target_folder_path)

    with zipfile.ZipFile(zip_filepath, 'r') as zip_ref:
        zip_ref.extractall(target_folder_path)

zip_filepath = 'images_zipped.zip'
unzip_to_cwd(zip_filepath)

In [3]:
path_to_data = './train'

In [4]:
train_data=utils.image_dataset_from_directory(
    path_to_data,
    labels="inferred",
    label_mode="int",
    validation_split=0.1,
    subset="training",
    shuffle=True,
    color_mode="rgb",
    image_size=(256,256),
    batch_size=64,
    seed=40,
)

Found 1129 files belonging to 10 classes.
Using 1017 files for training.


In [5]:
val_data=utils.image_dataset_from_directory(
    path_to_data,
    labels="inferred",
    label_mode="int",
    validation_split=0.1,
    subset="validation",
    color_mode="rgb",
    image_size=(256,256),
    batch_size=64,
    seed=40,
)

Found 1129 files belonging to 10 classes.
Using 112 files for validation.


In [6]:
for images,lables in train_data.take(1):
  print('Amount of train images: ', images.shape)
  print('Amount of train labels: ', lables.shape)

Amount of train images:  (64, 256, 256, 3)
Amount of train labels:  (64,)


In [7]:
for images,lables in val_data.take(1):
  print('Amount of validation images: ', images.shape)
  print('Amount of validation labels: ', lables.shape)

Amount of validation images:  (64, 256, 256, 3)
Amount of validation labels:  (64,)


In [8]:
classes=train_data.class_names
print('The following classes are in the dataset:')
print(classes)

The following classes are in the dataset:
['Avulsion fracture', 'Comminuted fracture', 'Fracture Dislocation', 'Greenstick fracture', 'Hairline Fracture', 'Impacted fracture', 'Longitudinal fracture', 'Oblique fracture', 'Pathological fracture', 'Spiral Fracture']


In [9]:
def normalize(image, label):
  return image/255.0, label

In [10]:
train_data = train_data.map(normalize)
val_data= val_data.map(normalize)

In [11]:
train_x=[]
train_y=[]

for image,label in train_data:
   train_x.append(image)
   train_y.append(label)

train_x = tf.concat(train_x, axis=0)
train_y = tf.concat(train_y, axis=0)

2024-08-20 08:59:54.720232: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 799801344 exceeds 10% of free system memory.


In [12]:
val_x=[]
val_y=[]

for image,label in train_data:
   val_x.append(image)
   val_y.append(label)

val_x = tf.concat(val_x, axis=0)
val_y = tf.concat(val_y, axis=0)

2024-08-20 08:59:56.729228: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 799801344 exceeds 10% of free system memory.


In [13]:
num_classes = 10
train_y = tf.keras.utils.to_categorical(train_y, num_classes=num_classes)
val_y = tf.keras.utils.to_categorical(val_y, num_classes=num_classes)

In [14]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,Dropout
from tensorflow.keras.optimizers import Adam

In [15]:
model = Sequential()
model.add(Conv2D(60, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(120, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(40, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 60)      1680      
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 60)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 120)     64920     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 62, 62, 120)       0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 461280)            0         
                                                                 
 dense (Dense)               (None, 40)                1

In [17]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
batch_size_param = int(os.getenv('batch_size'))
nr_of_epochs_param = int(os.getenv('nr_of_epochs'))

In [18]:
model.fit(train_x, train_y, batch_size=batch_size_param, epochs=nr_of_epochs_param,
          validation_data=(val_x,val_y))

Epoch 1/5
102/102 [==============================] - 203s 2s/step - loss: 2.5171 - accuracy: 0.1337 - val_loss: 2.2560 - val_accuracy: 0.1465
Epoch 2/5
102/102 [==============================] - 214s 2s/step - loss: 2.2100 - accuracy: 0.1986 - val_loss: 1.9715 - val_accuracy: 0.4503
Epoch 3/5
102/102 [==============================] - 212s 2s/step - loss: 1.7891 - accuracy: 0.4199 - val_loss: 1.1670 - val_accuracy: 0.6509
Epoch 4/5
102/102 [==============================] - 217s 2s/step - loss: 1.1349 - accuracy: 0.6490 - val_loss: 0.7733 - val_accuracy: 0.8387
Epoch 5/5
102/102 [==============================] - 217s 2s/step - loss: 0.6032 - accuracy: 0.8289 - val_loss: 0.3247 - val_accuracy: 0.9253


In [19]:
loss, accuracy = model.evaluate(val_x,val_y)
print('Test accuracy:', accuracy)

2024-08-20 09:17:58.554784: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 495482880 exceeds 10% of free system memory.


 1/32 [..............................] - ETA: 42s - loss: 0.1893 - accuracy: 0.9688

2024-08-20 09:17:59.890563: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 495482880 exceeds 10% of free system memory.


 2/32 [>.............................] - ETA: 40s - loss: 0.3415 - accuracy: 0.9531

2024-08-20 09:18:01.236977: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 495482880 exceeds 10% of free system memory.


32/32 [==============================] - 43s 1s/step - loss: 0.3247 - accuracy: 0.9253
Test accuracy: 0.9252703785896301


In [20]:
model.save('fracture_model.h5')

/opt/app-root/lib64/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
